In [45]:
# -*- coding: utf-8 -*-
# Copyright (C) 2018.02.09 kyung seok jeong <humanist96@koscom.co.kr>

from natto import MeCab
import pandas as pd
import collections
import re

In [46]:
def run_ma(text, nBest=1):
    """
    Return the dataframe of all Information of morpheme analyzer.
    - input : string, {nbest number}
    - output : dataframe
    """
    options=r'-F%m,%f[0],%f[1],%f[2],%f[3],%f[4],%f[5],%f[6],%f[7]\n'
    options+=" -N"+str(nBest)
    
    _me=MeCab(options)
    
    _df = pd.DataFrame(None, columns=['surface', 'tag', 'meaning_class', 'final_consonant', 
                                     'reading', 'type', 'first_tag', 'final_tag','expression'])

    i=0
    for term_str in str(_me.parse(text)).split('\n'):
        term_list = re.split(',', term_str)
        if len(term_list) < 2:
            continue
        _df.loc[i]=term_list   
        i+=1
    return _me, _df

me, df=run_ma("대한민국 형태소분석기는 한글 이 잘 쪼개질까?")
print(me)
print(df)
help(run_ma)

<natto.mecab.MeCab model=<cdata 'mecab_model_t *' 0x3ae2410>, tagger=<cdata 'mecab_t *' 0x3c9a110>, lattice=<cdata 'mecab_lattice_t *' 0x3cc1960>, libpath="/usr/local/lib/libmecab.so", options={'nbest': 1, 'node_format': '%m,%f[0],%f[1],%f[2],%f[3],%f[4],%f[5],%f[6],%f[7]\\n'}, dicts=[<natto.dictionary.DictionaryInfo dictionary=<cdata 'mecab_dictionary_info_t *' 0x3c90550>, filepath="/usr/local/lib/mecab/dic/mecab-ko-dic/sys.dic", charset=UTF-8, type=0>], version=0.996/ko-0.9.2>
  surface    tag meaning_class final_consonant reading      type first_tag  \
0    대한민국    NNP            지명               T    대한민국  Compound             
1     형태소    NNG                             F     형태소  Compound             
2     분석기    NNG                             F     분석기  Compound             
3       는     JX                             T       는                       
4      한글    NNG                             T      한글                       
5       이     MM           ~명사               F  

In [47]:
print(df)

def get_all_morph(df):
    """
    Returns all morphemes and Part-of-Speech.
    - input : dataframe
    - output : string
    """
    ret=''
    for index, row in df.iterrows():      
        if row['type'] == 'Inflect' or row['type'] == 'Compound':
            tag=row['expression']
            ret+=tag.replace('+',' ').replace("/*", '')+" "
        else:
            tag=row['tag']
            ret+=row['surface']+"/"+tag+" "        
    ret=ret.rstrip()
    ret=ret+"\n"
    
    return(ret)

get_all_morph(df)


  surface    tag meaning_class final_consonant reading      type first_tag  \
0    대한민국    NNP            지명               T    대한민국  Compound             
1     형태소    NNG                             F     형태소  Compound             
2     분석기    NNG                             F     분석기  Compound             
3       는     JX                             T       는                       
4      한글    NNG                             T      한글                       
5       이     MM           ~명사               F       이                       
6       잘    MAG                             T       잘                       
7      쪼개  VV+EC                             F      쪼개   Inflect        VV   
8      질까  VX+EF                             F      질까   Inflect        VX   
9       ?     SF                                                             

  final_tag         expression  
0            대한/NNG/*+민국/NNG/*  
1             형태/NNG/*+소/NNG/*  
2             분석/NNG/*+기/NNG/*  
3        

'대한/NNG 민국/NNG 형태/NNG 소/NNG 분석/NNG 기/NNG 는/JX 한글/NNG 이/MM 잘/MAG 쪼개/VV 어/EC 지/VX ᆯ까/EF ?/SF\n'

In [48]:
print(df)

def get_noun_morph(df, option='N'):
    """
    Returns noun morphemes and Part-of-Speech.
    - input : dataframe, {option : compound noun decomposition flag, default : N}
    - output : string
    """
    _noun_type = ['NNG', 'NNP']
    ret=''
    
    for index, row in df.iterrows():
        if row['tag'] in _noun_type:
            if row['type'] == 'Compound' and option != 'N':
                tag=row['expression']
                ret+=tag.replace('+',' ').replace("/*", '')+" "
            else:
                ret+=row['surface']+"/"+row['tag']+" " 
    ret=ret.rstrip()
    ret=ret+"\n"
    
    return(ret)

get_noun_morph(df)

  surface    tag meaning_class final_consonant reading      type first_tag  \
0    대한민국    NNP            지명               T    대한민국  Compound             
1     형태소    NNG                             F     형태소  Compound             
2     분석기    NNG                             F     분석기  Compound             
3       는     JX                             T       는                       
4      한글    NNG                             T      한글                       
5       이     MM           ~명사               F       이                       
6       잘    MAG                             T       잘                       
7      쪼개  VV+EC                             F      쪼개   Inflect        VV   
8      질까  VX+EF                             F      질까   Inflect        VX   
9       ?     SF                                                             

  final_tag         expression  
0            대한/NNG/*+민국/NNG/*  
1             형태/NNG/*+소/NNG/*  
2             분석/NNG/*+기/NNG/*  
3        

'대한민국/NNP 형태소/NNG 분석기/NNG 한글/NNG\n'

In [50]:
def get_noun_term_freq(df, option='N'):
    """
    Returns noun morphemes and freqeuncy
    - input : dataframe, {option : compound noun decomposition flag, default : N}
    - output : list of tuples(morpheme, frequency)
    """
    _noun_type = ['NNG', 'NNP']
    _terms = []
    
    for index, row in df.iterrows():
        if row['tag'] in _noun_type:
            if row['type'] == 'Compound' and option != 'N':
                tag=row['expression']
                _terms.extend(re.split(' ', tag.replace('+',' ').replace("/*", '')))
            else:
                _terms.append(row['surface'])
                
    return sorted(collections.Counter(_terms).items(), key=lambda x: x[1], reverse=True)

get_noun_term_freq(df)

[('분석기', 1), ('형태소', 1), ('한글', 1), ('대한민국', 1)]